# **Paul The Octopus 2022**

In [3]:
import pandas as pd
import google

from google.cloud import aiplatform
from google.cloud import bigquery

## ELO rating formulas

In [2]:
import numpy as np

continental_championships = ['AFC Asian Cup','African Cup of Nations','Copa América','CONCACAF Championship','Oceania Nations Cup','UEFA Euro']
qualification_championships = ['FIFA World Cup qualification','AFC Asian Cup qualification','African Nations Championship qualification','Copa América qualification','CONCACAF Championship qualification','Oceania Nations Cup qualification','UEFA Euro qualification']

#tournament weight
def k_factor(tournament):
    k = 30
    if tournament == 'Friendly':
        k = 20
    elif tournament in qualification_championships:
        k = 25
    elif tournament in continental_championships:
        k = 50
    elif tournament == 'FIFA World Cup':
        k = 60
    return k

#goal difference weight
def g_factor(goals_a, goals_b):
    g = 1
    n = abs(goals_a - goals_b)
    if n == 2:
        g = 1.5
    elif n > 2:
        g = (11 + n) / 8
    return g

def expected_result(rating_a, rating_b, neutral):
    dr = rating_a - rating_b
    if not neutral:
      dr = (rating_a + 100) - rating_b
    we = (1 / (10**(-dr / 400) + 1))
    return [np.round(we, 3), 1-np.round(we, 3)]

def actual_result(goals_a, goals_b):
    if goals_a < goals_b:
        sa = 0
        sb = 1
    elif goals_a > goals_b:
        sa = 1
        sb = 0
    elif goals_a == goals_b:
        sa = 0.5
        sb = 0.5
    return [sa, sb]

def calculate_elo_rating(rating_a, rating_b, goals_a, goals_b, tournament, neutral):
    
    k = k_factor(tournament)
    g = g_factor(goals_a, goals_b)
    wa, wb = actual_result(goals_a, goals_b)
    wea, web = expected_result(rating_a, rating_b, neutral)

    new_rating_a = rating_a + k*g*(wa - wea)
    new_rating_b = rating_b + k*g*(wb - web)

    return new_rating_a.round(0), new_rating_b.round(0)

## Read data from BigQuery

In [4]:
credentials, project_id = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient = bigquery.Client(credentials=credentials, project=project_id)

In [9]:
query = "SELECT * FROM phoenix-cit.paul_2022.historical_results"
historical_raw = bqclient.query(query).result().to_dataframe();

In [10]:
query = "SELECT * FROM phoenix-cit.paul_2022.ranking"
ranking_raw = bqclient.query(query).result().to_dataframe();

## Data cleanup and exploring

In [12]:
historical_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        44060 non-null  object 
 1   home_team   44060 non-null  object 
 2   away_team   44060 non-null  object 
 3   home_score  44059 non-null  float64
 4   away_score  44059 non-null  float64
 5   tournament  44060 non-null  object 
 6   city        44060 non-null  object 
 7   country     44060 non-null  object 
 8   neutral     44060 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


In [21]:
ranking_raw = ranking_raw[ranking_raw["rank_date"] != "rank_date"]

In [22]:
ranking_raw["rank_date"] = pd.to_datetime(ranking_raw["rank_date"], format='%Y-%m-%d')
ranking_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63916 entries, 0 to 63916
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   rank             63916 non-null  object        
 1   country_full     63916 non-null  object        
 2   country_abrv     63916 non-null  object        
 3   total_points     63916 non-null  object        
 4   previous_points  63916 non-null  object        
 5   rank_change      63916 non-null  object        
 6   confederation    63916 non-null  object        
 7   rank_date        63916 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 4.4+ MB


In [23]:
historical_raw["date"] = pd.to_datetime(historical_raw["date"])
historical_raw = historical_raw.replace({'home_team' : {
    'United States' : 'USA'
  }}
)
historical_raw = historical_raw.replace({'away_team' : {
    'United States' : 'USA'
  }}
)
historical_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        44060 non-null  datetime64[ns]
 1   home_team   44060 non-null  object        
 2   away_team   44060 non-null  object        
 3   home_score  44059 non-null  float64       
 4   away_score  44059 non-null  float64       
 5   tournament  44060 non-null  object        
 6   city        44060 non-null  object        
 7   country     44060 non-null  object        
 8   neutral     44060 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), object(5)
memory usage: 2.7+ MB


In [25]:
hist_unique_teams = pd.DataFrame(pd.unique((historical_raw['home_team']).append(historical_raw['away_team'])), columns=['Team'])
hist_unique_teams

,Team
0,Palestine
1,Cambodia
2,China PR
3,North Vietnam
4,North Korea
...,...
311,Crimea
312,Asturias
313,Surrey
314,Two Sicilies


In [27]:
rank_unique_teams = pd.DataFrame(pd.unique(ranking_raw['country_full']), columns=['country_full'])
rank_unique_teams

,country_full
0,Afghanistan
1,Anguilla
2,Albania
3,Algeria
4,Andorra
...,...
219,Serbia and Montenegro
220,São Tomé and Príncipe
221,Czechoslovakia
222,Türkiye


In [29]:
diff = pd.merge(hist_unique_teams, rank_unique_teams, left_on='Team', right_on='country_full', how='outer').query('Team != country_full')
diff.to_csv('store/diff.csv')
diff

cp: cannot create regular file '/content/drive/MyDrive/Paul the Octopus 2022/diff.csv': No such file or directory


,Team,country_full
3,North Vietnam,NaN
4,North Korea,NaN
5,Găgăuzia,NaN
6,Kyrgyzstan,NaN
7,Greenland,NaN
...,...,...
334,NaN,Zaire
335,NaN,Cabo Verde
336,NaN,Serbia and Montenegro
337,NaN,Türkiye


# Fixing country names

In [31]:
ranking_raw = ranking_raw.replace({'country_full' : {
    'Korea Republic' : 'South Korea',
    'Cabo Verde' : 'Cape Verde',
    'Cape Verde Islands' : 'Cape Verde',
    'Congo DR' : 'DR Congo',
    "Côte d'Ivoire" : 'Ivory Coast',
    'Curacao' : 'Curaçao',
    'FYR Macedonia' : 'North Macedonia',
    'IR Iran' : 'Iran',
    'Korea DPR' : 'North Korea',
    'Kyrgyz Republic' : 'Kyrgyzstan',
    'Sao Tome e Principe' : 'São Tomé and Príncipe	',
    'São Tomé e Príncipe' : 'São Tomé and Príncipe	',
    'St. Kitts and Nevis' : 'Saint Kitts and Nevis',
    'St. Lucia' : 'Saint Lucia',
    'St. Vincent / Grenadines' : 'Saint Vincent and the Grenadines',
    'St. Vincent and the Grenadines' : 'Saint Vincent and the Grenadines',
    'Swaziland' : 'Eswatini',
    'Türkiye' : 'Turkey',
    'US Virgin Islands' : 'United States Virgin Islands',
    'Zaire' : 'DR Congo'
  }}
)
ranking_raw

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,173,Afghanistan,AFG,48,102,-15,AFC,2006-07-12
1,190,Afghanistan,AFG,18,43,11,AFC,2007-04-18
2,182,Afghanistan,AFG,53,11,-14,AFC,2008-06-04
3,174,Afghanistan,AFG,61,10,-21,AFC,2011-04-13
4,179,Afghanistan,AFG,73,76,13,AFC,2011-07-27
...,...,...,...,...,...,...,...,...
63912,169,Saint Vincent and the Grenadines,VIN,117,88,-10,CONCACAF,2017-08-10
63913,192,United States Virgin Islands,VIR,17,0,-10,CONCACAF,2008-04-09
63914,78,Zambia,ZAM,418,365,-10,CAF,2014-10-23
63915,41,Zimbabwe,ZIM,38,35,-10,CAF,1995-02-20


## Merging Ranking data with Historical Results

In [36]:
historical_raw = historical_raw.sort_values(by="date")
ranking_raw = ranking_raw.sort_values(by="rank_date")

In [38]:
historical_home_rank = pd.merge_asof(historical_raw,ranking_raw, left_by='home_team', right_by='country_full',left_on='date', right_on='rank_date', direction='backward', allow_exact_matches=True)
historical_home_rank['home_rank'] = historical_home_rank['rank']
historical_home_rank['home_total_points'] = historical_home_rank['total_points']
historical_home_rank.drop(['rank','total_points','country_full','country_abrv','previous_points','rank_change','confederation','rank_date'], axis=1, inplace=True)
historical_home_rank

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rank,home_total_points
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,NaN,NaN
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,NaN,NaN
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,NaN,NaN
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,NaN,NaN
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
44055,2022-09-27,Switzerland,Czech Republic,2.0,1.0,UEFA Nations League,St. Gallen,Switzerland,False,16,1621.43
44056,2022-09-27,Burkina Faso,Comoros,2.0,1.0,Friendly,Casablanca,Morocco,True,55,1425.64
44057,2022-09-27,DR Congo,Sierra Leone,3.0,0.0,Friendly,Casablanca,Morocco,True,72,1332.84
44058,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False,49,1441.45


In [40]:
historical_rank = pd.merge_asof(historical_home_rank,ranking_raw, left_by='away_team', right_by='country_full',left_on='date', right_on='rank_date', direction='backward', allow_exact_matches=True)
historical_rank['away_rank'] = historical_rank['rank']
historical_rank['away_total_points'] = historical_rank['total_points']
historical_rank.drop(['rank','total_points','country_full','country_abrv','previous_points','rank_change','confederation','rank_date'], axis=1, inplace=True)
historical_rank

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rank,home_total_points,away_rank,away_total_points
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,NaN,NaN,NaN,NaN
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,NaN,NaN,NaN,NaN
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,NaN,NaN,NaN,NaN
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,NaN,NaN,NaN,NaN
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44055,2022-09-27,Switzerland,Czech Republic,2.0,1.0,UEFA Nations League,St. Gallen,Switzerland,False,16,1621.43,32,1502.9
44056,2022-09-27,Burkina Faso,Comoros,2.0,1.0,Friendly,Casablanca,Morocco,True,55,1425.64,127,1127.44
44057,2022-09-27,DR Congo,Sierra Leone,3.0,0.0,Friendly,Casablanca,Morocco,True,72,1332.84,113,1169.71
44058,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False,49,1441.45,58,1399.1


In [42]:
historical_rank = historical_rank[historical_rank['home_score'].notna()]
historical_rank = historical_rank[historical_rank['away_score'].notna()]

## Calculating ELO Ratings

In [48]:
current_elo = {}
for idx, row in historical_rank.iterrows():
    
    a_team = row['home_team']
    b_team = row['away_team']
    
    if a_team not in current_elo.keys():
        current_elo[a_team] = 2000
    
    if b_team not in current_elo.keys():
        current_elo[b_team] = 2000
    
    new_elo_a, new_elo_b = calculate_elo_rating(current_elo[a_team], current_elo[b_team], row['home_score'], row['away_score'], row['tournament'], row['neutral'])

    historical_rank.loc[idx,'k_factor'] = k_factor(row['tournament'])
    historical_rank.loc[idx,'home_elo_after'] = new_elo_a
    historical_rank.loc[idx,'away_elo_after'] = new_elo_b 
    historical_rank.loc[idx,'home_elo_before'] = current_elo[a_team]
    historical_rank.loc[idx,'away_elo_before'] = current_elo[b_team]

    current_elo[a_team] = new_elo_a
    current_elo[b_team] = new_elo_b

In [49]:
latest_elo = pd.DataFrame.from_dict(current_elo, orient='index', columns=['Elo'])
latest_elo.to_csv('store/latest_elo.csv')

In [50]:
historical_rank['score_diff'] = (historical_rank['home_score'] - historical_rank['away_score'])
historical_rank.to_csv('store/historical_rank.csv', index=False)

## Creating dummies for country values

In [51]:
home_team_dummies = pd.get_dummies(historical_rank['home_team'], prefix='dummy_home')
away_team_dummies = pd.get_dummies(historical_rank['away_team'], prefix='dummy_away')
historical_rank_dummies = historical_rank.join(home_team_dummies)
historical_rank_dummies = historical_rank_dummies.join(away_team_dummies)
historical_rank_dummies

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rank,...,dummy_away_Western Sahara,dummy_away_Yemen,dummy_away_Yemen DPR,dummy_away_Ynys Môn,dummy_away_Yorkshire,dummy_away_Yugoslavia,dummy_away_Zambia,dummy_away_Zanzibar,dummy_away_Zimbabwe,dummy_away_Åland Islands
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44054,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False,106,...,0,0,0,0,0,0,0,0,0,0
44055,2022-09-27,Switzerland,Czech Republic,2.0,1.0,UEFA Nations League,St. Gallen,Switzerland,False,16,...,0,0,0,0,0,0,0,0,0,0
44056,2022-09-27,Burkina Faso,Comoros,2.0,1.0,Friendly,Casablanca,Morocco,True,55,...,0,0,0,0,0,0,0,0,0,0
44057,2022-09-27,DR Congo,Sierra Leone,3.0,0.0,Friendly,Casablanca,Morocco,True,72,...,0,0,0,0,0,0,0,0,0,0


In [58]:
historical_rank_dummies.to_csv('store/historical_rank_elo_dummies.csv', index=False)

In [59]:
historical_rank_elo_dummies = historical_rank_dummies

# Training ML Model

In [55]:
from sklearn.model_selection import train_test_split

In [60]:
dummy_columns = [k for k in historical_rank_elo_dummies.columns if (k[:6]=='dummy_')]
predictors = ["k_factor","home_elo_before","away_elo_before","home_rank","away_rank","home_total_points","away_total_points","neutral"] + dummy_columns
len(predictors)

625

In [61]:
historical_rank_elo_dummies = historical_rank_elo_dummies.dropna()

In [62]:
target = historical_rank_elo_dummies[["home_score","away_score"]]
features = historical_rank_elo_dummies[predictors]

In [63]:
X_train,X_test,y_train,y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [66]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

params = {
    "n_estimators": 200,
    "max_depth": 6,
    "min_samples_split": 5,
    "learning_rate": 0.1,
    "loss": "squared_error",
}

gbr = MultiOutputRegressor(GradientBoostingRegressor(**params));
gbr.fit(X_train, y_train)

score = gbr.score(X_train, y_train)
print("Training score:", score)

score = gbr.score(X_test, y_test)
print("Test score:", score)

predictions = gbr.predict(X_test)

# Just ELO + Full history
# Training score: 0.6105493749837039
# Test score: 0.1995738325346768

# ELO + Rank + Points + Partial history rank doesn't go far into the past
# Training score: 0.4972803263176074
# Test score: 0.27695701519816174

# Only Would Cup countries data
# Training score: 0.5833660931368544
# Test score: 0.16928229496056335

Training score: 0.4972803263176074
Test score: 0.27695701519816174


In [86]:
from joblib import dump, load
dump(gbr, 'trained-model.joblib') 

['trained-model.joblib']

In [67]:
predicts = pd.DataFrame(predictions, columns=["pred_home_score","pred_away_score"])
test = X_test.join(y_test).reset_index(drop=True)
test[predicts.columns] = predicts

In [68]:
test[['home_elo_before','away_elo_before','home_rank','away_rank','home_total_points','away_total_points','neutral','home_score','away_score','pred_home_score','pred_away_score']].round({'pred_home_score': 0, 'pred_away_score': 0})

,home_elo_before,away_elo_before,home_rank,away_rank,home_total_points,away_total_points,neutral,home_score,away_score,pred_home_score,pred_away_score
0,1899.0,2328.0,127,40,207,735,False,0.0,4.0,1.0,2.0
1,2193.0,2048.0,48,83,677,399,False,1.0,2.0,2.0,1.0
2,2273.0,2140.0,23,31,49,46,True,2.0,4.0,1.0,1.0
3,1890.0,1868.0,147,143,185,189,True,1.0,1.0,1.0,1.0
4,1841.0,1883.0,165,155,111,140,False,2.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4862,2459.0,2519.0,4,3,1355,1505,False,1.0,0.0,1.0,1.0
4863,1938.0,2024.0,106,76,1205,1319,False,2.0,1.0,1.0,1.0
4864,1981.0,1885.0,73,122,520,391,False,4.0,2.0,2.0,1.0
4865,2267.0,2188.0,42,57,632,542,False,2.0,2.0,2.0,1.0


## Prepare matches dataset to run prediction

In [69]:
matches = pd.read_csv("store/matchs-schedule.csv", sep=";")

In [70]:
group_matches = matches[matches['phase'] == 'group matches']
group_matches = group_matches[['match','date','country1','coutry2']]
group_matches = group_matches.rename(columns={"country1": "home_team", "coutry2": "away_team"})
group_matches['date'] = pd.to_datetime(group_matches["date"])
group_matches = group_matches.sort_values(by='date')

In [71]:
group_matches_home_rank = pd.merge_asof(group_matches,ranking_raw, left_by='home_team', right_by='country_full',left_on='date', right_on='rank_date', direction='backward', allow_exact_matches=True)
group_matches_home_rank['home_rank'] = group_matches_home_rank['rank']
group_matches_home_rank['home_total_points'] = group_matches_home_rank['total_points']
group_matches_home_rank.drop(['rank','total_points','country_full','country_abrv','previous_points','rank_change','confederation','rank_date'], axis=1, inplace=True)

In [72]:
group_matches_rank = pd.merge_asof(group_matches_home_rank,ranking_raw, left_by='away_team', right_by='country_full',left_on='date', right_on='rank_date', direction='backward', allow_exact_matches=True)
group_matches_rank['away_rank'] = group_matches_rank['rank']
group_matches_rank['away_total_points'] = group_matches_rank['total_points']
group_matches_rank.drop(['rank','total_points','country_full','country_abrv','previous_points','rank_change','confederation','rank_date'], axis=1, inplace=True)

In [74]:
for idx, row in group_matches_rank.iterrows():
    a_team = row['home_team']
    b_team = row['away_team']
    
    group_matches_rank.loc[idx,'k_factor'] = 60
    group_matches_rank.loc[idx,'home_elo_before'] = current_elo[a_team]
    group_matches_rank.loc[idx,'away_elo_before'] = current_elo[b_team]
    group_matches_rank.loc[idx,'neutral'] = (a_team != 'Qatar') & (b_team != 'Qatar')

## Create dummies for country values

In [75]:
home_team_dummies = pd.get_dummies(group_matches_rank['home_team'], prefix='dummy_home')
away_team_dummies = pd.get_dummies(group_matches_rank['away_team'], prefix='dummy_away')
group_matches_dummies = group_matches_rank.join(home_team_dummies)
group_matches_dummies = group_matches_dummies.join(away_team_dummies)
group_matches_dummies

,match,date,home_team,away_team,home_rank,home_total_points,away_rank,away_total_points,k_factor,home_elo_before,...,dummy_away_Saudi Arabia,dummy_away_Senegal,dummy_away_Serbia,dummy_away_South Korea,dummy_away_Spain,dummy_away_Switzerland,dummy_away_Tunisia,dummy_away_USA,dummy_away_Uruguay,dummy_away_Wales
0,44,2022-01-12,Costa Rica,Germany,49,1437.43,12,1648.33,60.0,2284.0,...,0,0,0,0,0,0,0,0,0,0
1,43,2022-01-12,Japan,Spain,26,1531.53,7,1704.75,60.0,2346.0,...,0,0,0,0,1,0,0,0,0,0
2,42,2022-01-12,Canada,Morocco,40,1462.32,28,1529.93,60.0,2327.0,...,0,0,0,0,0,0,0,0,0,0
3,41,2022-01-12,Croatia,Belgium,15,1620.74,1,1828.45,60.0,2427.0,...,0,0,0,0,0,0,0,0,0,0
4,48,2022-02-12,Cameroon,Brazil,38,1480.82,2,1823.42,60.0,2176.0,...,0,0,0,0,0,0,0,0,0,0
5,46,2022-02-12,South Korea,Portugal,29,1522.85,8,1660.25,60.0,2366.0,...,0,0,0,0,0,0,0,0,0,0
6,45,2022-02-12,Ghana,Uruguay,61,1381.45,16,1614.05,60.0,2138.0,...,0,0,0,0,0,0,0,0,1,0
7,47,2022-02-12,Serbia,Switzerland,25,1547.38,14,1642.83,60.0,2387.0,...,0,0,0,0,0,1,0,0,0,0
8,1,2022-11-21,Qatar,Ecuador,50,1439.89,44,1464.39,60.0,2223.0,...,0,0,0,0,0,0,0,0,0,0
9,4,2022-11-21,USA,Wales,16,1627.48,19,1569.82,60.0,2356.0,...,0,0,0,0,0,0,0,0,0,1


In [76]:
col_list = historical_rank_elo_dummies.columns.difference(group_matches_dummies.columns).tolist()
for col in col_list:
    group_matches_dummies[col] = 0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until


In [77]:
dummy_columns = [k for k in group_matches_dummies.columns if (k[:6]=='dummy_')]
predictors_b = ["k_factor","home_elo_before","away_elo_before","home_rank","away_rank","home_total_points","away_total_points","neutral"] + dummy_columns
len(predictors_b)

625

## Run prediction

In [87]:
gbr = load('trained-model.joblib') 
predictions = gbr.predict(group_matches_dummies[predictors])

score_predictions = pd.DataFrame(predictions, columns=["pred_home_score","pred_away_score"])
results = group_matches_dummies.reset_index(drop=True)
results[predicts.columns] = score_predictions

In [88]:
results = results[['match','date','home_team','away_team','home_elo_before','away_elo_before','home_rank','away_rank','home_total_points','away_total_points','neutral','pred_home_score','pred_away_score']]
results = results.rename(columns = {'home_team':'home', 'away_team':'away', 'pred_home_score':'home_score', 'pred_away_score':'away_score'})
results = results.sort_values(by='match').reset_index(drop=True)

In [89]:
results

,match,date,home,away,home_elo_before,away_elo_before,home_rank,away_rank,home_total_points,away_total_points,neutral,home_score,away_score
0,1,2022-11-21,Qatar,Ecuador,2223.0,2364.0,50,44,1439.89,1464.39,False,0.906116,1.303322
1,2,2022-11-21,Senegal,Netherlands,2323.0,2537.0,18,8,1584.38,1694.51,True,0.802285,1.554281
2,3,2022-11-21,England,Iran,2461.0,2394.0,5,20,1728.47,1564.61,True,1.216232,1.250199
3,4,2022-11-21,USA,Wales,2356.0,2295.0,16,19,1627.48,1569.82,True,1.107780,0.978261
4,5,2022-11-22,France,Australia,2537.0,2296.0,4,38,1759.78,1488.72,True,1.713458,0.815707
5,6,2022-11-22,Denmark,Tunisia,2474.0,2266.0,10,30,1666.57,1507.54,True,1.622041,1.042367
6,7,2022-11-22,Mexico,Poland,2385.0,2316.0,13,26,1644.89,1548.59,True,1.318464,1.158769
7,8,2022-11-22,Argentina,Saudi Arabia,2650.0,2210.0,3,51,1773.88,1437.78,True,2.985828,0.723949
8,9,2022-11-23,Belgium,Canada,2556.0,2327.0,2,41,1816.71,1475.0,True,2.509902,0.874451
9,10,2022-11-23,Spain,Costa Rica,2556.0,2284.0,7,31,1715.22,1503.59,True,1.849682,0.847021


## Export predictions

In [90]:
export = results[['home','home_score','away_score','away']].round({'home_score': 0, 'away_score': 0})
export = export.astype({'home_score': 'int32','away_score': 'int32' })
export

,home,home_score,away_score,away
0,Qatar,1,1,Ecuador
1,Senegal,1,2,Netherlands
2,England,1,1,Iran
3,USA,1,1,Wales
4,France,2,1,Australia
5,Denmark,2,1,Tunisia
6,Mexico,1,1,Poland
7,Argentina,3,1,Saudi Arabia
8,Belgium,3,1,Canada
9,Spain,2,1,Costa Rica


In [83]:
export.to_csv('submit.csv', index=False)